In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame


from numpy import dstack

import copy

import os
from glob import glob

from path import Path

from numpy import mean
from numpy import std

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Bidirectional

from keras.models import model_from_json


from keras.callbacks import EarlyStopping

from keras.optimizers import Adam

import random

from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

from matplotlib import pyplot


## Load model

In [4]:
# load json and create model
json_file = open('model-0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model-0.h5")
print("Loaded model from disk")

Loaded model from disk


## Load samples

In [27]:
def pad_with_zeros(orig_ndarray, desired_rows_count):
    (frames, features) = orig_ndarray.shape

    sample_df = pd.DataFrame(data=orig_ndarray,
                             index=np.arange(frames),
                             columns=np.arange(features))

    zeros_df = pd.DataFrame(0,
                            index=np.arange(desired_rows_count),
                            columns=np.arange(features),
                            dtype='float')

    for i in range(features):
        zeros_df[i] = sample_df[i].astype(float)
    
    
    padded_df = zeros_df.fillna(0)

    return padded_df.to_numpy()

def filter_active_keypoints(frame_df):
    #active_keypoints = [0, 1, 8, 2, 3, 4, 5, 6, 7] # trunk
    
    active_keypoints = [0, 1, 8, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14]
    
    filtered_df  = frame_df[frame_df.index.isin(active_keypoints)]
    #filtered_df = frame_df

    return filtered_df

def get_frame_data(filepath):
    frame_data = pd.read_csv(filepath)
    
    x_data = filter_active_keypoints(frame_data.iloc[:, 0])
    y_data = filter_active_keypoints(frame_data.iloc[:, 1])
    
    #print(f"x_data shape: {x_data.shape}")
    #print(f"y_data shape: {y_data.shape}")
        
    h_stacked = np.hstack((x_data, y_data))
    #h_stacked = np.hstack((y_data))
    
    return h_stacked

def get_frame_file_path(frame_file_path_template, frame_idx):
    frame_file_path = frame_file_path_template.replace("[frame_idx]", str(frame_idx))
    return frame_file_path

def get_frames_count(root_path, sample_dir_name):
    return len([Path(f).abspath() for f in glob(f"{root_path}/{sample_dir_name}" + '/*')])

def get_frames(root_path, sample_dir_name):
    frames = []
    for frame_idx in range(0, get_frames_count(root_path, sample_dir_name)):
        frame_file_path_template = f"{root_path}/{sample_dir_name}/{sample_dir_name}.mov-[frame_idx]-0.csv"
        frame_file_path = get_frame_file_path(frame_file_path_template, frame_idx)
        frame_data = get_frame_data(frame_file_path)
        
        frames.append(frame_data)
        
    return np.dstack(frames)

def get_samples_list(sample_dir_names, root_path):
    samples = []
    sample_names = []
    for sample_dir_name in sample_dir_names:
        frames = get_frames(root_path, sample_dir_name)
        squeezed = np.squeeze(frames)
        axes_swapped = np.swapaxes(squeezed, 0, 1)
        samples.append(axes_swapped)
        
        sample_names.append(sample_dir_name)
        
    return samples, sample_names

root_path = "/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints"

sample_dir_names = [n for n in os.listdir(root_path) if os.path.isdir(f"{root_path}/{n}")]

samples, sample_names = get_samples_list(sample_dir_names, root_path)

def get_padded_samples(samples):
    padded_samples_list = []

    for idx, sample_ndarray in enumerate(samples):
        desired_rows_count = 110

        padded_ndarray = pad_with_zeros(sample_ndarray, desired_rows_count)

        padded_samples_list = padded_samples_list + [padded_ndarray]
        
    return padded_samples_list

padded_samples_list = get_padded_samples(samples)

padded_samples_beefy_list = padded_samples_list

print(f"padded samples len: {len(padded_samples_beefy_list)}")
padded_samples_beefy_ndarray = np.asarray(padded_samples_beefy_list)

padded samples len: 113


## Load labels

In [24]:
def get_label(sample_dir_name):
    return 0 if sample_dir_name[0] == 'b' else 1

def get_y_labels(sample_dir_names):
    return [get_label(l) for l in sample_dir_names]

y_labels = get_y_labels(sample_dir_names) # classifier labels, where 0 = backflip and 1 = flack

y_labels_stacked = np.dstack(y_labels)
print(f"y_labels_stacked shape: {y_labels_stacked.shape}")

y_labels_categorical = to_categorical(y_labels_stacked) 
print(f"y_labels_categorical shape: {y_labels_categorical.shape}")

y_labels_squeezed = np.squeeze(y_labels_categorical)
print(f"y_labels_squeezed shape {y_labels_squeezed.shape}")

(y_rows, y_cols) = y_labels_squeezed.shape
y_labels_list = [[y_labels_squeezed[i, 0], y_labels_squeezed[i, 1]] for i in range(y_rows)]

y_labels_beefy_ndarray = np.asarray(y_labels_list)

y_labels_stacked shape: (1, 1, 113)
y_labels_categorical shape: (1, 1, 113, 2)
y_labels_squeezed shape (113, 2)


In [17]:
padded_samples_beefy_ndarray.shape

(113, 110, 30)

## Predict

In [28]:
ynew = loaded_model.predict_classes(padded_samples_beefy_ndarray)
# show the inputs and predicted outputs
for i in range(len(padded_samples_beefy_ndarray)):
    pred_y = ynew[i]
    actual_y = y_labels[i]
    
    same = False
    if pred_y == actual_y:
        same = True
    
    print("Name=%s, X=%s, Predicted=%s, Actual=%s, same=%s" % (sample_names[i], i, pred_y, actual_y, same))

Name=backflip-40-margus, X=0, Predicted=0, Actual=0, same=True
Name=flack-31-rasmus, X=1, Predicted=1, Actual=1, same=True
Name=flack-19-rasmus, X=2, Predicted=1, Actual=1, same=True
Name=flack-59-martin, X=3, Predicted=1, Actual=1, same=True
Name=backflip-66-allar, X=4, Predicted=0, Actual=0, same=True
Name=flack-55-martin, X=5, Predicted=1, Actual=1, same=True
Name=flack-68-rasmus, X=6, Predicted=1, Actual=1, same=True
Name=backflip-23-tiit, X=7, Predicted=0, Actual=0, same=True
Name=flack-7-hendrik, X=8, Predicted=1, Actual=1, same=True
Name=flack-4-martin, X=9, Predicted=1, Actual=1, same=True
Name=flack-15-rasmus, X=10, Predicted=1, Actual=1, same=True
Name=flack-36-hendrik, X=11, Predicted=1, Actual=1, same=True
Name=backflip-64-allar, X=12, Predicted=0, Actual=0, same=True
Name=backflip-6-rasmus, X=13, Predicted=0, Actual=0, same=True
Name=flack-82-martin, X=14, Predicted=1, Actual=1, same=True
Name=flack-35-margus, X=15, Predicted=1, Actual=1, same=True
Name=backflip-38-mario, 

## Activations

In [62]:
import keras.backend as K


# 3rd layer is LSTM layer with output shape (Batch_Size, 512)
lstm = loaded_model.layers[0]

print(lstm)

# Get output from intermediate layer to visualize activations
attn_func = K.function(inputs = [lstm.input, K.learning_phase()],
                       outputs = [lstm.output])



In [58]:

# get html element
def cstr(s, color='black'):
    if s == ' ':
        return "<text style=color:#000;padding-left:10px;background-color:{}> </text>".format(color, s)
    else:
        return "<text style=color:#000;background-color:{}>{} </text>".format(color, s)

# print html
def print_color(t):
    display(html_print(''.join([cstr(ti, color=ci) for ti,ci in t])))

# get appropriate color for value
def get_clr(value):
    colors = ['#85c2e1', '#89c4e2', '#95cae5', '#99cce6', '#a1d0e8'
        '#b2d9ec', '#baddee', '#c2e1f0', '#eff7fb', '#f9e8e8',
        '#f9e8e8', '#f9d4d4', '#f9bdbd', '#f8a8a8', '#f68f8f',
        '#f47676', '#f45f5f', '#f34343', '#f33b3b', '#f42e2e']
    value = int((value * 100) / 5)
    return colors[value]

# sigmoid function
def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

In [67]:
def getOutputLayer(layerNumber, model, X):
    return K.function([model.layers[0].input, K.learning_phase()],
                      [model.layers[layerNumber].output])([X])

def visualize(output_values, result_list, cell_no):
    print("\nCell Number:", cell_no, "\n")
    text_colours = []
    for i in range(len(output_values)):
        text = (result_list[i], get_clr(output_values[i][cell_no]))
        text_colours.append(text)
    print_color(text_colours)

# Get Predictions from random sequence
def get_predictions(data, sample_names, sample_nr, model):
    pattern = data[sample_nr]
    sample_name = sample_names[sample_nr]
    
    print(f"Analyzing activations for {sample_name}")
    
    result_list, output_values = [], []
    
    pattern_rows, pattern_columns = pattern.shape
    for row in pattern:
        print("\"" + ' '.join(["{:.0f}".format(cell) for cell in row]) + "\"")

    (rows_count, features_count) = pattern.shape
    pattern = pattern.reshape(1, rows_count, features_count)

    #for i in range(1000):

        # Prediction
    prediction = model.predict(pattern, verbose=0)
    print(f"Prediction: {prediction}")

        # LSTM Activations
    output = getOutputLayer(0, model, pattern)
    print(output)
    
    output = sigmoid(output)
    output_values.append(output)

        # Predicted Character
    #    index = np.argmax(prediction)
     #   result = int_to_char[index]

        # Preparing input for next character
      #  seq_in = [int_to_char[value] for value in pattern]
       # pattern.append(index)
        #pattern = pattern[1:len(pattern)]

        # Saving generated characters
       # result_list.append(result)
    return output_values, result_list

output_values, result_list = get_predictions(padded_samples_beefy_ndarray, sample_names, 30, loaded_model)

print(output_values)

for cell_no in [0, 1]:
    visualize(output_values, result_list, cell_no)

Analyzing activations for flack-17-rasmus
"-38 26 9 -100 -227 38 -115 -233 0 -20 -12 3 12 3 15 -262 -218 -215 -194 -221 -230 -209 -227 0 0 165 336 -0 165 350"
"-35 29 24 -112 -229 35 -115 -235 0 -15 -9 3 9 3 9 -262 -215 -215 -191 -209 -230 -194 -212 -0 -0 168 336 -0 165 350"
"-34 28 17 -103 -226 37 -114 -234 0 -18 -12 3 9 3 11 -262 -216 -215 -189 -206 -229 -197 -209 -0 0 167 336 -0 165 350"
"-30 28 21 -102 -223 38 -110 -234 0 -17 -12 3 7 3 9 -262 -215 -215 -181 -187 -227 -185 -189 -0 0 167 335 -0 165 350"
"-27 29 22 -97 -216 40 -104 -230 -0 -17 -13 3 5 2 9 -262 -215 -214 -173 -166 -223 -177 -167 0 1 166 335 0 165 350"
"-26 32 27 -95 -205 40 -97 -223 0 -13 -11 3 4 1 9 -262 -215 -213 -165 -144 -220 -168 -144 0 1 166 335 0 166 350"
"-25 37 31 -90 -194 40 -92 -213 0 -11 -12 3 3 0 9 -262 -215 -213 -156 -123 -218 -158 -122 0 1 167 335 0 167 351"
"-24 41 38 -79 -180 38 -84 -201 0 -6 -10 3 2 -0 9 -262 -215 -214 -144 -103 -217 -149 -101 1 0 169 335 1 169 352"
"-24 43 44 -71 -159 36 -77 -186 0 -

ValueError: Arguments and signature arguments do not match. got: 4, expected: 5 